In [8]:
#Working on pyspark in Jupyterlab
import pyspark

In [2]:
from pyspark.sql import SparkSession 

In [1]:
from pyspark import SparkConf, SparkContext

In [4]:
conf = SparkConf(). \
    setMaster("local"). \
    setAppName("Orders_Revenue"). \
    set("conf.ui.port","10567")

In [5]:
sc = SparkContext(conf=conf)

22/11/21 08:44:48 WARN Utils: Your hostname, codeStation resolves to a loopback address: 127.0.1.1; using 192.168.213.83 instead (on interface wlo1)
22/11/21 08:44:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/21 08:44:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
spark = SparkSession.builder.appName('Practice').getOrCreate()

22/11/21 06:01:17 WARN Utils: Your hostname, codeStation resolves to a loopback address: 127.0.1.1; using 192.168.213.83 instead (on interface wlo1)
22/11/21 06:01:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/21 06:01:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Execution of the file in command prompt

spark-submit --master local --deploy-mode client --conf spark.ui.port=12901 --conf spark.dynamicAllocation.anabled=false --num-executors 1 --executor-memory 512M --executor-cores 1 file_name_for_DE.py

In [7]:
spark.sparkContext

<SparkContext master=local[*] appName=Practice>

In [9]:
sc = spark.sparkContext

NameError: name 'spark' is not defined

In [10]:
productPath = "/home/solverbot/spark-warehouse/retail_db/products/part-00000"
orderitemPath = "/home/solverbot/spark-warehouse/retail_db/order_items/part-00000"
ordersPath = "/home/solverbot/spark-warehouse/retail_db/orders/part-00000.txt"

In [11]:
products = sc.textFile(productPath)

In [12]:
help(products.flatMap)

Help on method flatMap in module pyspark.rdd:

flatMap(f: Callable[[~T], Iterable[~U]], preservesPartitioning: bool = False) -> 'RDD[U]' method of pyspark.rdd.RDD instance
    Return a new RDD by first applying a function to all elements of this
    RDD, and then flattening the results.
    
    Examples
    --------
    >>> rdd = sc.parallelize([2, 3, 4])
    >>> sorted(rdd.flatMap(lambda x: range(1, x)).collect())
    [1, 1, 1, 2, 2, 3]
    >>> sorted(rdd.flatMap(lambda x: [(x, x), (x, x)]).collect())
    [(2, 2), (2, 2), (3, 3), (3, 3), (4, 4), (4, 4)]



In [11]:
type(products)

pyspark.rdd.RDD

In [13]:
for i in products.take(5):
    print(i)

1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy
2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat
3,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat
4,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat
5,2,Riddell Youth Revolution Speed Custom Footbal,,199.99,http://images.acmesports.sports/Riddell+Youth+Revolution+Speed+Custom+Football+Helmet


In [14]:
import getpass
username = getpass.getuser()

In [15]:
username

'solverbot'

In [ ]:
spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Data Processing - Overview'). \
    master('yarn'). \
    getOrCreate()b


In [16]:
newSessio = SparkSession.builder.config('spark.ui.port','0').\
            config("spark.sql.warehouse.dir",f"/home/{username}/spark-warehouse/retail_db/"). \
            appName(f'{username} | Python - Data Processing - Overview'). \
            getOrCreate()

22/11/21 06:08:57 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [17]:
newSessio

In [19]:
orders = sc.textFile(ordersPath)
orders.take(5)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE']

In [24]:
orderFiltered = orders.filter(lambda o: o.split(',')[3] in ('COMPLETE','CLOSED'))
orderFiltered.takeSample(True,2)

['37551,2014-03-13 00:00:00.0,3384,COMPLETE',
 '52013,2014-06-16 00:00:00.0,7642,COMPLETE']

In [25]:
ordersMap = orders.map(lambda o: (int(o.split(',')[0]), o.split(',')[1]))
ordersMap.take(2)

[(1, '2013-07-25 00:00:00.0'), (2, '2013-07-25 00:00:00.0')]

In [26]:
orderItems = sc.textFile(orderitemPath)
orderItems.count()

172198

In [30]:
orderItemsMap = orderItems.map(lambda oi: (int(oi.split(',')[1]),float(oi.split(',')[4])))
orderItemsMap.take(5)

[(1, 299.98), (2, 199.99), (2, 250.0), (2, 129.99), (4, 49.98)]

In [31]:
orderItemsReduce = orderItemsMap.reduceByKey(lambda o,p: o + p)
orderItemsReduce.take(2)

[(2, 579.98), (4, 699.85)]

In [33]:
help(orderItemsMap.groupByKey)

Help on method groupByKey in module pyspark.rdd:

groupByKey(numPartitions: Optional[int] = None, partitionFunc: Callable[[~K], int] = <function portable_hash at 0x7fca45489480>) -> 'RDD[Tuple[K, Iterable[V]]]' method of pyspark.rdd.PipelinedRDD instance
    Group the values for each key in the RDD into a single sequence.
    Hash-partitions the resulting RDD with numPartitions partitions.
    
    Notes
    -----
    If you are grouping in order to perform an aggregation (such as a
    sum or average) over each key, using reduceByKey or aggregateByKey will
    provide much better performance.
    
    Examples
    --------
    >>> rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
    >>> sorted(rdd.groupByKey().mapValues(len).collect())
    [('a', 2), ('b', 1)]
    >>> sorted(rdd.groupByKey().mapValues(list).collect())
    [('a', [1, 1]), ('b', [1])]



In [37]:
orderItemsGBK = orderItemsMap.groupByKey()
for i in orderItemsGBK.take(2): print(f'{i[0]}=>{list(i[1])}')

2=>[199.99, 250.0, 129.99]
4=>[49.98, 299.95, 150.0, 199.92]


In [38]:
orderJoin = ordersMap.join(orderItemsReduce)
orderJoin.take(5)

[(4, ('2013-07-25 00:00:00.0', 699.85)),
 (8, ('2013-07-25 00:00:00.0', 729.8399999999999)),
 (12, ('2013-07-25 00:00:00.0', 1299.8700000000001)),
 (16, ('2013-07-25 00:00:00.0', 419.93)),
 (20, ('2013-07-25 00:00:00.0', 879.8599999999999))]

In [39]:
orderJoinList = ordersMap.join(orderItemsGBK)
orderJoinList.take(5)

[(4,
  ('2013-07-25 00:00:00.0',
   <pyspark.resultiterable.ResultIterable at 0x7fca28369450>)),
 (8,
  ('2013-07-25 00:00:00.0',
   <pyspark.resultiterable.ResultIterable at 0x7fca2836ad10>)),
 (12,
  ('2013-07-25 00:00:00.0',
   <pyspark.resultiterable.ResultIterable at 0x7fca28368e50>)),
 (16,
  ('2013-07-25 00:00:00.0',
   <pyspark.resultiterable.ResultIterable at 0x7fca2836ab60>)),
 (20,
  ('2013-07-25 00:00:00.0',
   <pyspark.resultiterable.ResultIterable at 0x7fca2836bf40>))]

In [51]:
list(orderJoinList.take(1)[0][1][1])

[49.98, 299.95, 150.0, 199.92]

In [52]:
orderJoinSorted = orderJoin.sortByKey()
orderJoinSorted.take(2)

[(1, ('2013-07-25 00:00:00.0', 299.98)),
 (2, ('2013-07-25 00:00:00.0', 579.98))]

In [53]:
help(orderJoin.sortBy)

Help on method sortBy in module pyspark.rdd:

sortBy(keyfunc: Callable[[~T], ForwardRef('S')], ascending: bool = True, numPartitions: Optional[int] = None) -> 'RDD[T]' method of pyspark.rdd.PipelinedRDD instance
    Sorts this RDD by the given keyfunc
    
    Examples
    --------
    >>> tmp = [('a', 1), ('b', 2), ('1', 3), ('d', 4), ('2', 5)]
    >>> sc.parallelize(tmp).sortBy(lambda x: x[0]).collect()
    [('1', 3), ('2', 5), ('a', 1), ('b', 2), ('d', 4)]
    >>> sc.parallelize(tmp).sortBy(lambda x: x[1]).collect()
    [('a', 1), ('b', 2), ('1', 3), ('d', 4), ('2', 5)]



In [55]:
orderJoinSortedRevn = orderJoin.sortBy(lambda x: x[1][1], ascending=False)
orderJoinSortedRevn.take(1)

[(68703, ('2013-08-16 00:00:00.0', 3449.9100000000003))]

In [58]:
orderJoinListPop = orderJoinList.map(lambda x: (x[0], x[1][0], list(x[1][1])))
orderJoinListPop.take(1)

[(4, '2013-07-25 00:00:00.0', [49.98, 299.95, 150.0, 199.92])]

In [61]:
#writing to file
orderJoinListPop.saveAsTextFile(f"/home/{username}/orders_and_sales")

In [62]:
readingBack = sc.textFile(f"/home/{username}/orders_and_sales")
readingBack.count()

57431

In [63]:
readingBack.take(2)

["(7, '2013-07-25 00:00:00.0', [199.99, 299.98, 79.95])",
 "(11, '2013-07-25 00:00:00.0', [59.99, 159.96, 49.98, 399.96, 249.9])"]